<a href="https://colab.research.google.com/github/fetterollie/rec-systems-project/blob/main/Jasmine_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

insert header

# Title

## Business Understanding and Business Problem

### Cost of Errors

In [4]:
# Mount google dirve

from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

## Data Understanding

In [5]:
# Install surprise
!pip install scikit-surprise

#imports grabbed from lecture
from random import gauss as gs, uniform as uni, seed
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
anceled
ERROR: Operation cancelled by user


### Initial Data

In [6]:
#the four csv files provided
links_df = pd.read_csv('/content/gdrive/MyDrive/rec-systems-project/data/links.csv')
movies_df = pd.read_csv('/content/gdrive/MyDrive/rec-systems-project/data/movies.csv')
ratings_df = pd.read_csv('/content/gdrive/MyDrive/rec-systems-project/data/ratings.csv')
tags_df = pd.read_csv('/content/gdrive/MyDrive/rec-systems-project/data/tags.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/rec-systems-project/data/links.csv'

In [ ]:
links_df.head()

In [ ]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


# Jasmine's edit:
From ratings_df create a new dataset without timestmap, using surprise

In [ ]:
# Drop timestamp

new_rating_df= ratings_df.drop(columns='timestamp')
new_rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
# Transform the dataset into something compatible with surprise

from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(new_rating_df,reader)

In [ ]:
# Check how many users and items

dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


### Find the best model

In [1]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [2]:
# Perform a gridsearch with SVD

params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}
g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(data)

NameError: name 'data' is not defined

In [3]:
print(g_s_svd.best_score)
print(g_s_svd.best_params)

AttributeError: 'GridSearchCV' object has no attribute 'best_score'

In [ ]:
# cross validating with KNNBasic
knn_basic = KNNBasic(sim_options={'name':'pearson', 'user_based':True})
cv_knn_basic = cross_validate(knn_basic, data, n_jobs=-1)

In [ ]:
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.97531727, 0.98181778, 0.96914813, 0.9693987 , 0.97106524]))
('test_mae', array([0.75366721, 0.754633  , 0.74890637, 0.74957363, 0.74948928]))
('fit_time', (0.2631371021270752, 0.18639683723449707, 0.22644448280334473, 0.19980430603027344, 0.11818456649780273))
('test_time', (3.361798048019409, 2.8813493251800537, 1.8045036792755127, 1.6815376281738281, 0.832385778427124))
-----------------------
0.9733494229185327


In [ ]:
# cross validating with KNNBaseline
knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
cv_knn_baseline = cross_validate(knn_baseline,data)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [ ]:
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.8722948 , 0.87912393, 0.87622948, 0.87940505, 0.87706492]))
('test_mae', array([0.66568865, 0.67370377, 0.66938545, 0.6698581 , 0.66902346]))
('fit_time', (0.2979440689086914, 0.33266353607177734, 0.30804014205932617, 0.28485918045043945, 0.3089728355407715))
('test_time', (1.5576658248901367, 1.4377005100250244, 1.554227352142334, 1.445190191268921, 1.5927979946136475))


0.8768236344603892

### Making simple predictions

In [ ]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [ ]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0391242707606394, details={'was_impossible': False})

### Obtaining User Ratings

In [ ]:
def movie_rater(movies_df, num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movies_df[movies_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movies_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list      

In [ ]:
user_rating = movie_rater(movies_df, 4, 'Comedy')

      movieId                          title        genres
6591    55280  Lars and the Real Girl (2007)  Comedy|Drama
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3
      movieId            title                  genres
5548    26680  Cry-Baby (1990)  Comedy|Musical|Romance
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                      title  genres
2699     3622  Twelve Chairs, The (1970)  Comedy
How do you rate this movie on a scale of 1-5, press n if you have not seen :
n
      movieId                                  title  \
8597   117646  Dragonheart 2: A New Beginning (2000)   

                                              genres  
8597  Action|Adventure|Comedy|Drama|Fantasy|Thriller  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4
     movieId                   title                  genres
643      829  Joe's Apartment (1996)  Comedy|Fantasy|Musical
How do you ra

In [ ]:
user_rating

[{'userId': 1000, 'movieId': 55280, 'rating': '3'},
 {'userId': 1000, 'movieId': 117646, 'rating': '4'},
 {'userId': 1000, 'movieId': 829, 'rating': '3'},
 {'userId': 1000, 'movieId': 76293, 'rating': '4'}]

### Making Predictions With the New Ratings

In [ ]:
## add the new ratings to the original ratings DataFrame

new_ratings_df = new_rating_df.append(user_rating, ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df, reader)

In [ ]:
# train a model using the new combined DataFrame

svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [ ]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)

list_of_movies = []
for m_id in new_rating_df['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(1000,m_id)[3]))

In [ ]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)

In [ ]:
# return the top n recommendations using the 
def recommended_movies(user_ratings, movie_title_df, n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies, movies_df, 5)

Recommendation #  1 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 

Recommendation #  2 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  3 :  975    Cool Hand Luke (1967)
Name: title, dtype: object 

Recommendation #  4 :  2226    Fight Club (1999)
Name: title, dtype: object 

Recommendation #  5 :  906    Lawrence of Arabia (1962)
Name: title, dtype: object 



## Exploratory Data Analysis

## Modeling

## Final Model

## Results/Conclusions

### Further Exploration/Questions